In [59]:
# Imports

import gspread as gs
import pandas as pd
import numpy as np

In [ ]:
# Match Results Google Sheet
sheet_id = "1RBWOGn_cBeK9qQmSbVz7NE_XGTBwevMVOFWbW9EHGf8"
sheet_name = "Match_Results_2"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
results_df = pd.read_csv(url).fillna('')
results_df.head(50)

In [ ]:
# Match Results Google Sheet
# gc = gs.service_account(filename='shuffleboard-325614-64ddad20cd94.json')
# sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1RBWOGn_cBeK9qQmSbVz7NE_XGTBwevMVOFWbW9EHGf8/edit#gid=1110865845')
# ws = sh.worksheet('Match_Results_2')


In [ ]:
# Creating Won_Loss Results Table
winners = dict(enumerate(results_df['Match Winner']))
losers = dict(enumerate(results_df['Match Loser']))
winners.update(losers)
players = winners
wins = results_df['player1'].value_counts()
losses = results_df['player2'].value_counts()
w_l = pd.concat([wins,losses],axis=1).fillna(0).astype(int).round().rename(columns={'player1':'Wins','player2':'Losses'})
w_l.loc[:,'Matches']=w_l.sum(numeric_only=True, axis=1)
w_l['Win%%'] = w_l['Wins']/w_l['Matches']
w_l['Player']=w_l.index
column_names = ['Player',"Matches", "Wins", "Losses", "Win%"]
w_l['Win%'] = w_l['Win%'].round(3)
w_l = w_l.reindex(columns=column_names)
w_l = w_l.reset_index(drop=True)
w_l.head(5)

In [ ]:
# Rankings
rankings_df = pd.read_csv('Rankings.csv')
rankings_df.drop_duplicates(subset=['Player'],inplace=True)
rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.loc[:,'ELO']=1600
initial_ranking = 1600
k_factor = 32
rankings_df.head(20)

In [ ]:
# ELO Formula
def elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor):
    R1 = 10**(p1_ranking/400)
    R2 = 10**(p2_ranking/400)
    exp_1 = R1/(R1+R2)
    exp_2 = R2/(R1+R2)
    ELO_1 = p1_ranking+k_factor*(p1_score-exp_1)
    ELO_2 = p2_ranking+k_factor*(p2_score-exp_2)
    return ELO_1, ELO_2

In [ ]:
#this loop reads the scores in order starting from the top of the matches
#every time you run this loop it will read all of the scores again
for idx in results_df.index:
    player_1 = results_df.player1[idx]
    p1_score = results_df.player1score[idx]
    player_2 = results_df.player2[idx]
    p2_score = results_df.player2score[idx]
    current_rankings_list = list(set(rankings_df.index))
    if player_1 in current_rankings_list:
        p1_ranking = rankings_df.at[player_1,'ELO']
    else:
        p1_ranking = initial_ranking
    if player_2 in current_rankings_list:
        p2_ranking = rankings_df.at[player_2,'ELO']
    else:
        p2_ranking = initial_ranking
    
    ELO_1, ELO_2 = elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor)
    
    if player_1 in current_rankings_list:
        rankings_df.at[player_1,'ELO'] = ELO_1
    else:
        new_ELO_dict = {'Player':player_1,'ELO':ELO_1}
        rankings_df = rankings_df.append(new_ELO_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
    if player_2 in current_rankings_list:
        rankings_df.at[player_2,'ELO'] = ELO_2
    else:
        new_ELO_dict = {'Player':player_2,'ELO':ELO_2}
        rankings_df = rankings_df.append(new_ELO_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.reset_index(drop=True,inplace=True)
rankings_df.sort_values(by=['ELO'],inplace=True,ascending=False,ignore_index=True)
rankings_df.index+=1
rankings_df.to_csv('Rankings.csv',index=False)

In [ ]:
pd.set_option('display.max_rows', None)
rankings_df

In [ ]:
# Formatting and Printing Final Table
final_table = w_l.merge(rankings_df, how="left", on="Player")
final_table_print = final_table.drop('Matches', axis=1)
final_table_print['Ranking'] = final_table_print[["ELO","Win%"]].apply(tuple,axis=1)\
             .rank(ascending=False, method="min").astype(int)
column_names = ['Ranking','Player', "Wins", "Losses", "Win%","ELO"]
final_table_print = final_table_print.reindex(columns=column_names)
final_table_print = final_table_print.sort_values(by=['Ranking','Win%'],ascending=[True,False])
final_table_print.to_csv('Rankings_Table.csv',index=False)
final_table_print.to_html('Rankings_Table.html',index=False)
final_table_print